#### Use this for sentiment analysis with Logistic Regression and Naive Bayes
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/1_IMPORTANT_Capstone-Project-master/Capstone-Project-master/Capstone_Project_Sentiment_Analysis.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/1_IMPORTANT_Capstone-Project-master/Capstone-Project-master/Capstone_Project_KNN_RecommenderSystem.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/2_Amazon-Product-Recommender-System-master/Naive%20Bayes/Amazon%20reviews_Naive%20Bayes.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/2_Amazon-Product-Recommender-System-master/Recommender%20System/Recommender%20System.ipynb

Removing Contractions - https://medium.com/analytics-vidhya/sentiment-analysis-on-amazon-reviews-using-tf-idf-approach-c5ab4c36e7a1


http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Recommendation%2BSystem%2BNotebook_WIP.ipynb
http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Recommendation%2BSystem%2BNotebook.ipynb

http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Akash_collaborative_user_base..ipynb

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

In [2]:
import re

In [3]:
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn import neighbors
import matplotlib.pyplot as plt


In [4]:
df = pd.read_csv("sample30.csv")

In [5]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [6]:
print(df.columns)
print(df.shape)

Index(['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews_date',
       'reviews_didPurchase', 'reviews_doRecommend', 'reviews_rating',
       'reviews_text', 'reviews_title', 'reviews_userCity',
       'reviews_userProvince', 'reviews_username', 'user_sentiment'],
      dtype='object')
(30000, 15)


In [7]:
df.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [8]:
#Tokenize the review text to words
reg = re.compile('[^a-z]+')
def tokenizeReview(text):
    text = text.lower()
    text = reg.sub(' ', text).strip()
    return text

In [9]:
df['reviews_tokenized'] = df['reviews_text'].apply(tokenizeReview)

In [10]:
df.shape

(30000, 16)

In [11]:
df = df.drop(['reviews_text'], axis=1)

In [12]:
df.shape

(30000, 15)

In [13]:
df['reviews_tokenized'].head(5)

0    i love this album it s very good more to the h...
1    good flavor this review was collected as part ...
2                                          good flavor
3    i read through the reviews on here before look...
4    my husband bought this gel for us the gel caus...
Name: reviews_tokenized, dtype: object

In [14]:
df['user_sentiment'].value_counts()

Positive    26632
Negative     3367
Name: user_sentiment, dtype: int64

In [15]:
def senti(txt):
    if str(txt).lower() == 'positive':
        return 1
    else:
        return 0
    
X = df['reviews_tokenized']
y = df['user_sentiment'].apply(senti)

In [16]:
print(len(X))
print(len(y))

30000
30000


In [17]:
y

0        1
1        1
2        1
3        0
4        0
        ..
29995    1
29996    1
29997    1
29998    1
29999    1
Name: user_sentiment, Length: 30000, dtype: int64

In [18]:
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words= 'english')

In [19]:
def model_fit(X, y, feature_model,ml_model,coef_show=1):
    
    X_features = feature_model.fit_transform(X)
    print('# features: {}'.format(X_features.shape[1]))
    
#    tempX = feature_model.transform(['this is an awful product', 'i am satisfied with this. will come back again'])
    
    X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size = 0.2, random_state=43)
    clf = ml_model.fit(X_train, y_train)
    clf_pred = clf.predict(X_test)
    accuracy = clf.score(X_test, y_test)
    model_performance = classification_report(y_test, clf_pred)
    print ('accuracy of the model: ', accuracy)
    print('')
    print(model_performance)

#    print(X_train)
#    print("---")
#    print(X_test)
#    print("---")
#    print(dir(ml_model))
    
#    tempPred = ml_model.predict(tempX)
#    print('---Prediction----')
#    print(tempPred)
    
#    print(vars(X_test))
#    print("indices - ", len(X_test.indices))
#    print("indptr - ", len(X_test.indptr))
#    print("data - ", len(X_test.data))
    
#    print(dir(X_test))
#    print("X_ARRAY ", X_test._get_arrayXarray(0, 529))
    
    if coef_show == 1: 
        w = feature_model.get_feature_names()
        coef = clf.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : w, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print('Top 10 positive features (variables)')
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print('Top 10 negative features (variables)')        
        print(coeff_df.tail(20).to_string(index=False))
       
    #return X_test, clf_pred
    return feature_model, clf

featuModel, clasModel = model_fit(X, y, tfidf,LogisticRegression(),coef_show=1)


# features: 1000
accuracy of the model:  0.9221666666666667

              precision    recall  f1-score   support

           0       0.81      0.38      0.52       660
           1       0.93      0.99      0.96      5340

    accuracy                           0.92      6000
   macro avg       0.87      0.69      0.74      6000
weighted avg       0.92      0.92      0.91      6000


Top 10 positive features (variables)
      Word  Coefficient
     great    13.299884
      love    11.054383
      best     8.156580
      good     7.800511
      easy     7.735733
     clean     7.719841
    better     5.808192
     loved     5.688351
      nice     5.268984
 excellent     5.121848
   awesome     4.898837
   perfect     4.861898
  favorite     4.555234
   enjoyed     4.373794
   amazing     4.364436
      free     3.939587
 wonderful     3.916948
    really     3.575024
     handy     3.529437
     happy     3.373818

Top 10 negative features (variables)
          Word  Coefficient
    

In [20]:
#Do reviews classification as good or bad sentiment. 
Xfeatu = featuModel.transform(['this is an awful product', 'i am satisfied with this. will come back again'])
print(clasModel.predict(Xfeatu))

[0 1]


In [21]:
#model_fit(X, y, tfidf,LogisticRegression(),coef_show=1)

In [22]:
#print(xtest)
#print(pred) 

#### Recommendation System - User based collaborative

In [23]:
# Reading ratings file
ratings = pd.read_csv('sample30.csv' , encoding='latin-1')
ratings.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [24]:
ratings = ratings[['id', 'reviews_username', 'name', 'reviews_rating']]
ratings.head()

,id,reviews_username,name,reviews_rating
0,AV13O1A8GV-KLJ3akUyj,joshua,Pink Friday: Roman Reloaded Re-Up (w/dvd),5
1,AV14LG0R-jtxr-f38QfS,dorothy w,Lundberg Organic Cinnamon Toast Rice Cakes,5
2,AV14LG0R-jtxr-f38QfS,dorothy w,Lundberg Organic Cinnamon Toast Rice Cakes,5
3,AV16khLE-jtxr-f38VFn,rebecca,K-Y Love Sensuality Pleasure Gel,1
4,AV16khLE-jtxr-f38VFn,walker557,K-Y Love Sensuality Pleasure Gel,1


In [25]:
ratings.reviews_rating.unique()

array([5, 1, 3, 4, 2], dtype=int64)

In [26]:
#Number of users
lst = ratings.reviews_username.value_counts()
print(lst)
print(len(lst))
print(sum(lst))

byamazon customer    41
mike                 41
chris                32
lisa                 16
rick                 15
                     ..
star09                1
ingrid                1
peanutbrain           1
netcruzer7            1
honestreview          1
Name: reviews_username, Length: 24914, dtype: int64
24914
29937


In [27]:
ratings.reviews_rating.value_counts()

5    20831
4     6020
1     1384
3     1345
2      420
Name: reviews_rating, dtype: int64

In [28]:
print(ratings.shape)
print(ratings.id.value_counts())

(30000, 4)
AVpf3VOfilAPnD_xjpun    8545
AVpfPaoqLJeJML435Xk9    3325
AVpfJP1C1cnluZ0-e3Xy    2039
AVpfW8y_LJeJML437ySW    1186
AVpfRTh1ilAPnD_xYic2    1143
                        ... 
AVpe4Bq81cnluZ0-YwTN       1
AVpfDflP1cnluZ0-c5RJ       1
AVpfbjraLJeJML439F91       1
AVpfIN84LJeJML432_D9       1
AVpfshNsLJeJML43CB8q       1
Name: id, Length: 271, dtype: int64


### Dividing the dataset into train and test

In [29]:
# Test and Train split of the dataset.
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings, test_size=0.30, random_state=31)

In [30]:
print(train.shape)
print(test.shape)

(21000, 4)
(9000, 4)


In [31]:
#Proof that the default aggregate function for duplicates is mean
d = pd.read_csv('TestFile.csv' , encoding='latin-1')
print(d)
t = d.pivot_table(index='reviews_username', columns='name', values='reviews_rating')
t

   id   name  reviews_rating reviews_username
0   1  brush               3             shal
1   2  brush               4             shal
2   3    pen               5             shal
3   4    pen               5            kumar
4   5  paint               4             shal
5   6  paint               2            kumar
6   7  board               1             shal
7   8  board               2            kumar
8   9  brush               3             shal


name,board,brush,paint,pen
reviews_username,,,,
kumar,2.0,NaN,2.0,5.0
shal,1.0,3.333333,4.0,5.0


In [32]:
print(len(ratings.reviews_username.unique()))
print(len(ratings.name.unique()))

tmp = ratings.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(0)
tmp.head()

24915
271


name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz","42 Dual Drop Leaf Table with 2 Madrid Chairs""",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),...,Walkers Stem Ginger Shortbread,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Wilton Black Dots Standard Baking Cups,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02dakota,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02deuce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Pivot the train ratings' dataset into matrix format in which columns are items and the rows are user.
df_pivot = train.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(0)
df_pivot.head(5)

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Creating dummy train & dummy test dataset
These dataset will be used for prediction 
- Dummy train will be used later for prediction of the movies which has not been rated by the user. To ignore the movies rated by the user, we will mark it as 0 during prediction. The movies not rated by user is marked as 1 for prediction in dummy train dataset. 

- Dummy test will be used for evaluation. To evaluate, we will only make prediction on the movies rated by the user. So, this is marked as 1. This is just opposite of dummy_train.

In [34]:
# Copy the train dataset into dummy_train
dummy_train = train.copy()
dummy_train.head()

,id,reviews_username,name,reviews_rating
11155,AVpf3VOfilAPnD_xjpun,gor05751,Clorox Disinfecting Wipes Value Pack Scented 1...,5
22114,AVpf--o9LJeJML43G_Fq,annavsl,Vaseline Intensive Care Healthy Hands Stronger...,4
26218,AVpfPkrkilAPnD_xX-L9,bluenose,"Finish Gelpacs Dishwasher Detergent, Orange Sc...",5
18966,AVpfJP1C1cnluZ0-e3Xy,kiki,Clorox Disinfecting Bathroom Cleaner,5
29999,AVpfW8y_LJeJML437ySW,kcoopxoxo,L'or233al Paris Elvive Extraordinary Clay Reba...,5


In [35]:
print(dummy_train['reviews_rating'].value_counts())
print(dummy_train.shape)

5    14514
4     4234
1      995
3      957
2      300
Name: reviews_rating, dtype: int64
(21000, 4)


In [36]:
# The movies not rated by user is marked as 1 for prediction. 
dummy_train['reviews_rating'] = dummy_train['reviews_rating'].apply(lambda x: 0 if x>=1 else 1)

In [37]:
dummy_train.reviews_rating.unique()

array([0], dtype=int64)

In [38]:
dummy_train.head()

,id,reviews_username,name,reviews_rating
11155,AVpf3VOfilAPnD_xjpun,gor05751,Clorox Disinfecting Wipes Value Pack Scented 1...,0
22114,AVpf--o9LJeJML43G_Fq,annavsl,Vaseline Intensive Care Healthy Hands Stronger...,0
26218,AVpfPkrkilAPnD_xX-L9,bluenose,"Finish Gelpacs Dishwasher Detergent, Orange Sc...",0
18966,AVpfJP1C1cnluZ0-e3Xy,kiki,Clorox Disinfecting Bathroom Cleaner,0
29999,AVpfW8y_LJeJML437ySW,kcoopxoxo,L'or233al Paris Elvive Extraordinary Clay Reba...,0


In [39]:
# Convert the dummy train dataset into matrix format.
dummy_train = dummy_train.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(1)

In [40]:
dummy_train.shape

(18252, 252)

#### Checking for a given user if the reviewed items are set to 0 and rest to 1. All good so far.

In [41]:
print(dummy_train.loc['abby'].value_counts())
print(dummy_train.loc['abismomy'].value_counts())
print(dummy_train.loc['ac94'].value_counts())


1.0    249
0.0      3
Name: abby, dtype: int64
1.0    250
0.0      2
Name: abismomy, dtype: int64
1.0    250
0.0      2
Name: ac94, dtype: int64


In [42]:
dummy_train

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
00sab00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
01impala,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0325home,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
06stidriver,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zulaa118,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
zxcsdfd,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
zxjki,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


**Cosine Similarity**

Cosine Similarity is a measurement that quantifies the similarity between two vectors [Which is Rating Vector in this case] 

**Adjusted Cosine**

Adjusted cosine similarity is a modified version of vector-based similarity where we incorporate the fact that different users have different ratings schemes. In other words, some users might rate items highly in general, and others might give items lower ratings as a preference. To handle this nature from rating given by user , we subtract average ratings for each user from each user's rating for different movies.

# User Similarity Matrix

## Using Cosine Similarity
#### Pair-wise similarity is done row-wise in the pivot table

In [43]:
from sklearn.metrics.pairwise import pairwise_distances

# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_pivot, metric='cosine')
print(user_correlation)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


#### Check if the correlation values are set to meaningful values.

In [44]:
print(sum(user_correlation[0])/len(user_correlation[0]))
print(sum(user_correlation[1])/len(user_correlation[1]))
print(sum(user_correlation[2])/len(user_correlation[2]))
print(sum(user_correlation[3])/len(user_correlation[3]))
print(sum(user_correlation[4])/len(user_correlation[4]))
print(sum(user_correlation[5])/len(user_correlation[5]))
print(sum(user_correlation[6])/len(user_correlation[6]))
print(sum(user_correlation[7])/len(user_correlation[7]))


0.0007699409750198169
0.025349981632427198
0.11795404456236924
0.2965389464248795
0.2965389464248795
0.06681913443692394
0.02378234824574021
0.003206839857539405


In [45]:
user_correlation[np.isnan(user_correlation)] = 0

In [46]:
print(user_correlation)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [47]:
user_correlation.shape

(18252, 18252)

## Using adjusted Cosine 

### Here, we are not removing the NaN values and calculating the mean only for the movies rated by the user

In [48]:
# Create a user-item review matrix.
df_pivot = train.pivot_table(index='reviews_username', columns='name', values='reviews_rating')

In [49]:
df_pivot.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01impala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0325home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06stidriver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Normalising the rating of the movie for each user around 0 mean

In [50]:
#This does the mean for each row as the axis is 1
mean = np.nanmean(df_pivot, axis=1)

In [51]:
print(mean)
print(len(mean))

[4. 1. 3. ... 2. 5. 5.]
18252


In [52]:
df_subtracted = (df_pivot.T - mean).T

In [53]:
df_subtracted.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01impala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0325home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06stidriver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Finding cosine similarity

In [54]:
from sklearn.metrics.pairwise import pairwise_distances

In [55]:
# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Prediction - User User

Doing the prediction for the users which are positively related with other users, and not the users which are negatively related as we are interested in the users which are more similar to the current users. So, ignoring the correlation for values less than 0. 

In [56]:
user_correlation[user_correlation<0]=0
user_correlation

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [57]:
#Check if there are legitimate values in the correlation matrix
sum(sum(user_correlation))

10185.884430507153

Rating predicted by the user (for items rated as well as not rated) is the weighted sum of correlation with the item rating (as present in the rating dataset). 

In [58]:
user_predicted_ratings = np.dot(user_correlation, df_pivot.fillna(0))
user_predicted_ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
user_predicted_ratings.shape

(18252, 252)

In [60]:
sum(sum(user_predicted_ratings))

120324.65698632221

Since we are interested only in the movies not rated by the user, we will ignore the movies rated by the user by making it zero. 

In [61]:
user_final_rating = np.multiply(user_predicted_ratings,dummy_train)
user_final_rating.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Check for our algorithm correctness - Sum the numbers by items to see if anything meaningful is arrived at. For these items, check in the excel sheet and pick a user (who is in the train data set) and get recommendations for her.


In [62]:
print(user_final_rating.T.sum(axis=1).sort_values(ascending=False)[0:60])

name
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)                                        5404.701432
The Resident Evil Collection 5 Discs (blu-Ray)                                                             5196.472401
Mike Dave Need Wedding Dates (dvd + Digital)                                                               5117.180772
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd                                            4756.145677
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                                                       4343.492836
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                                                  4053.226287
Red (special Edition) (dvdvideo)                                                                           2982.062401
Clorox Disinfecting Bathroom Cleaner                                                                       2413.224154
Windex Original Glass Cleaner Refill 67.6oz

In [63]:
d = user_final_rating.loc['abby'].sort_values(ascending=False)[0:10]
d

name
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd        6.269380
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)    5.679494
Clorox Disinfecting Bathroom Cleaner                                   3.651637
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                   3.370093
The Resident Evil Collection 5 Discs (blu-Ray)                         3.370093
Tostitos Bite Size Tortilla Chips                                      3.370093
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total              3.370093
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo               3.370093
Chester's Cheese Flavored Puffcorn Snacks                              3.333333
Tostitos Simply Blue Corn Tortilla Chips                               3.333333
Name: abby, dtype: float64

### Finding the top 5 recommendation for the *user*

In [64]:
# Take the user ID as input.
user_input = input("Enter your user name")
print(user_input)

Enter your user nameabby
abby


In [65]:
user_final_rating.head(2)

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
d = user_final_rating.loc[user_input].sort_values(ascending=False)[0:20]
d

name
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd                                  6.269380
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)                              5.679494
Clorox Disinfecting Bathroom Cleaner                                                             3.651637
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                                             3.370093
The Resident Evil Collection 5 Discs (blu-Ray)                                                   3.370093
Tostitos Bite Size Tortilla Chips                                                                3.370093
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                                        3.370093
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo                                         3.370093
Chester's Cheese Flavored Puffcorn Snacks                                                        3.333333
Tostitos Simply Blue Corn Tortilla Chips 

### Evaluation - User User 

Evaluation will be same as you have seen above for the prediction. The only difference being, you will evaluate for the item already rated by the user insead of predicting it for the movie not rated by the user. 

In [67]:
# Find out the common users of test and train dataset.
common = test[test.reviews_username.isin(train.reviews_username)]
common.shape

(2049, 4)

In [68]:
common.head()

,id,reviews_username,name,reviews_rating
20634,AVpfkak01cnluZ0-nJj6,girlie,Starbucks Iced Expresso Classics Vanilla Latte...,5
26479,AVpfPPkEilAPnD_xX3cP,kasheville,Aveeno Baby Continuous Protection Lotion Sunsc...,1
15098,AVpf4oLxLJeJML43FcxC,arlene,Avery174 Ready Index Contemporary Table Of Con...,5
17968,AVpfcu821cnluZ0-k8ep,mimi3,Vaseline Intensive Care Lip Therapy Cocoa Butter,5
28020,AVpfRTh1ilAPnD_xYic2,movielover,Planes: Fire Rescue (2 Discs) (includes Digita...,5


In [69]:
# convert into the user-item matrix.
common_user_based_matrix = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating')

In [70]:
common_user_based_matrix.head()

name,100:Complete First Season (blu-Ray),"42 Dual Drop Leaf Table with 2 Madrid Chairs""",Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz","Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz","Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter",Axe Dry Anti-Perspirant Deodorant Invisible Solid Phoenix,"BRIDGESTONE 130/70ZR18M/C(63W)FRONT EXEDRA G851, CRUISER RADL",...,Various - Country's Greatest Gospel:Gold Ed (cd),Various - Red Hot Blue:Tribute To Cole Porter (cd),Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234asdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123charlie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# Convert the user_correlation matrix into dataframe.
user_correlation_df = pd.DataFrame(user_correlation)

In [72]:
user_correlation_df

,0,1,2,3,4,5,6,7,8,9,...,18242,18243,18244,18245,18246,18247,18248,18249,18250,18251
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
df_subtracted.head(1)

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
df_subtracted.index

Index(['00dog3', '00sab00', '01impala', '0325home', '06stidriver', '09mommy11',
       '1085', '10ten', '11111111aaaaaaaaaaaaaaaaa', '1143mom',
       ...
       'zozo0o', 'zpalma', 'zsazsa', 'zt313', 'zubb', 'zulaa118', 'zxcsdfd',
       'zxjki', 'zyiah4', 'zzdiane'],
      dtype='object', name='reviews_username', length=18252)

In [75]:
user_correlation_df['reviews_username'] = df_subtracted.index
user_correlation_df.set_index('reviews_username',inplace=True)
user_correlation_df.head()

,0,1,2,3,4,5,6,7,8,9,...,18242,18243,18244,18245,18246,18247,18248,18249,18250,18251
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
common.head(1)

,id,reviews_username,name,reviews_rating
20634,AVpfkak01cnluZ0-nJj6,girlie,Starbucks Iced Expresso Classics Vanilla Latte...,5


In [77]:
list_name = common.reviews_username.tolist()
user_correlation_df.columns = df_subtracted.index.tolist()
user_correlation_df_1 =  user_correlation_df[user_correlation_df.index.isin(list_name)]

In [78]:
len(list_name)

2049

In [79]:
user_correlation_df.columns

Index(['00dog3', '00sab00', '01impala', '0325home', '06stidriver', '09mommy11',
       '1085', '10ten', '11111111aaaaaaaaaaaaaaaaa', '1143mom',
       ...
       'zozo0o', 'zpalma', 'zsazsa', 'zt313', 'zubb', 'zulaa118', 'zxcsdfd',
       'zxjki', 'zyiah4', 'zzdiane'],
      dtype='object', length=18252)

In [80]:
user_correlation_df_1

,00dog3,00sab00,01impala,0325home,06stidriver,09mommy11,1085,10ten,11111111aaaaaaaaaaaaaaaaa,1143mom,...,zozo0o,zpalma,zsazsa,zt313,zubb,zulaa118,zxcsdfd,zxjki,zyiah4,zzdiane
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234asdf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123charlie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zach,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zapman,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zburt5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
user_correlation_df_1.shape

(1728, 18252)

In [82]:
user_correlation_df_2 = user_correlation_df_1.T[user_correlation_df_1.T.index.isin(list_name)]

In [83]:
user_correlation_df_3 = user_correlation_df_2.T

In [84]:
user_correlation_df_3.head()

,00sab00,1234,1234asdf,123charlie,1943,4cloroxl,4djs,7inthenest,85lisa,a hutch,...,ygtz,yoadrian,yohnie1,yummy,yvonne,zach,zapman,zburt5,zebras,zmom
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234asdf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123charlie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1943,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
user_correlation_df_3.shape

(1728, 1728)

In [86]:
user_correlation_df_3[user_correlation_df_3<0]=0

common_user_predicted_ratings = np.dot(user_correlation_df_3, common_user_based_matrix.fillna(0))
common_user_predicted_ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [87]:
dummy_test = common.copy()
dummy_test['reviews_rating'] = dummy_test['reviews_rating'].apply(lambda x: 1 if x>=1 else 0)
dummy_test = dummy_test.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(0)

In [88]:
dummy_test.shape

(1728, 126)

In [89]:
common_user_predicted_ratings = np.multiply(common_user_predicted_ratings,dummy_test)

In [90]:
common_user_predicted_ratings.head(2)

name,100:Complete First Season (blu-Ray),"42 Dual Drop Leaf Table with 2 Madrid Chairs""",Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz","Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz","Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter",Axe Dry Anti-Perspirant Deodorant Invisible Solid Phoenix,"BRIDGESTONE 130/70ZR18M/C(63W)FRONT EXEDRA G851, CRUISER RADL",...,Various - Country's Greatest Gospel:Gold Ed (cd),Various - Red Hot Blue:Tribute To Cole Porter (cd),Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Calculating the RMSE for only the movies rated by user. For RMSE, normalising the rating to (1,5) range.

In [91]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_user_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

MinMaxScaler(feature_range=(1, 5))
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


C:\Users\kumaraguru\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\kumaraguru\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [92]:
y.shape

(1728, 126)

In [93]:
common_ = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating')

In [94]:
common_

name,100:Complete First Season (blu-Ray),"42 Dual Drop Leaf Table with 2 Madrid Chairs""",Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz","Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz","Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter",Axe Dry Anti-Perspirant Deodorant Invisible Solid Phoenix,"BRIDGESTONE 130/70ZR18M/C(63W)FRONT EXEDRA G851, CRUISER RADL",...,Various - Country's Greatest Gospel:Gold Ed (cd),Various - Red Hot Blue:Tribute To Cole Porter (cd),Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234asdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123charlie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zapman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
zburt5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [96]:
print(total_non_nan)

314


In [97]:
rmse = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse)

2.498437231297542


## Using Item similarity

# Item Based Similarity

Taking the transpose of the rating matrix to normalize the rating around the mean for different item. In the user based similarity, we had taken mean for each user instead of each item. 

In [98]:
df_pivot = train.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T
df_pivot.head()

reviews_username,00dog3,00sab00,01impala,0325home,06stidriver,09mommy11,1085,10ten,11111111aaaaaaaaaaaaaaaaa,1143mom,...,zozo0o,zpalma,zsazsa,zt313,zubb,zulaa118,zxcsdfd,zxjki,zyiah4,zzdiane
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100:Complete First Season (blu-Ray),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2x Ultra Era with Oxi Booster, 50fl oz",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Normalising the movie rating for each movie for using the Adujsted Cosine

In [99]:
mean = np.nanmean(df_pivot, axis=1)
df_subtracted = (df_pivot.T-mean).T

In [100]:
df_subtracted.head()

reviews_username,00dog3,00sab00,01impala,0325home,06stidriver,09mommy11,1085,10ten,11111111aaaaaaaaaaaaaaaaa,1143mom,...,zozo0o,zpalma,zsazsa,zt313,zubb,zulaa118,zxcsdfd,zxjki,zyiah4,zzdiane
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100:Complete First Season (blu-Ray),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2x Ultra Era with Oxi Booster, 50fl oz",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finding the cosine similarity using pairwise distances approach

In [101]:
from sklearn.metrics.pairwise import pairwise_distances

In [102]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation)

[[ 1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          1.          0.         ... -0.00657044  0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.         -0.00657044  0.         ...  1.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          1.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   1.        ]]


Filtering the correlation only for which the value is greater than 0. (Positively correlated)

In [103]:
item_correlation[item_correlation<0]=0
item_correlation

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

# Prediction - Item Item

In [104]:
item_predicted_ratings = np.dot((df_pivot.fillna(0).T),item_correlation)
item_predicted_ratings

array([[0.        , 0.        , 0.        , ..., 0.02352264, 0.        ,
        0.        ],
       [0.        , 0.00117139, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04848007, 0.        , ..., 0.        , 0.        ,
        0.00338607],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00062229, 0.00063779,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00155571, 0.00159447,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.03252811, 0.02124867,
        0.        ]])

In [105]:
item_predicted_ratings.shape

(18252, 252)

In [106]:
dummy_train.shape

(18252, 252)

### Filtering the rating only for the movies not rated by the user for recommendation

In [107]:
item_final_rating = np.multiply(item_predicted_ratings,dummy_train)
item_final_rating.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.023523,0.000000,0.000000
00sab00,0.0,0.001171,0.0,0.0,0.0,0.0,0.0,0.0,0.003316,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
01impala,0.0,0.048480,0.0,0.0,0.0,0.0,0.0,0.0,0.014725,0.000000,...,0.0,0.0,0.0,0.006729,0.002859,0.0,0.0,0.000000,0.000000,0.003386
0325home,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
06stidriver,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000


### Finding the top 5 recommendation for the *user*

In [134]:
# Take the user ID as input
user_input = input("Enter your user name")
print(user_input)

Enter your user nameabby
abby


In [135]:
# Recommending the Top 5 products to the user.
d = item_final_rating.loc[user_input].sort_values(ascending=False)[0:20]
d

name
Chester's Cheese Flavored Puffcorn Snacks                                         0.546107
Windex Original Glass Cleaner Refill 67.6oz (2 Liter)                             0.167279
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo                          0.086751
Pleasant Hearth 1,800 sq ft Wood Burning Stove with Blower, Medium, LWS-127201    0.042787
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)               0.032291
Nexxus Exxtra Gel Style Creation Sculptor                                         0.030927
Aussie Aussome Volume Shampoo, 13.5 Oz                                            0.028226
Mike Dave Need Wedding Dates (dvd + Digital)                                      0.027196
Diet Canada Dry Ginger Ale - 12pk/12 Fl Oz Cans                                   0.023710
Dark Shadows (includes Digital Copy) (ultraviolet) (dvdvideo)                     0.021194
Cheetos Crunchy Flamin' Hot Cheese Flavored Snacks                                0.0

# Evaluation - Item Item

Evaluation will we same as you have seen above for the prediction. The only difference being, you will evaluate for the movie already rated by the user insead of predicting it for the movie not rated by the user. 

In [110]:
test.columns

Index(['id', 'reviews_username', 'name', 'reviews_rating'], dtype='object')

In [111]:
common =  test[test.name.isin(train.name)]
common.shape

(8978, 4)

In [112]:
common.head(4)

,id,reviews_username,name,reviews_rating
4332,AVpe9W4D1cnluZ0-avf0,sansname,Hoover174 Platinum Collection153 Lightweight B...,5
3434,AVpe5s3RLJeJML43x4eA,jennypstrong,Pantene Pro-V Expert Collection Age Defy Condi...,5
20634,AVpfkak01cnluZ0-nJj6,girlie,Starbucks Iced Expresso Classics Vanilla Latte...,5
26479,AVpfPPkEilAPnD_xX3cP,kasheville,Aveeno Baby Continuous Protection Lotion Sunsc...,1


In [113]:
common_item_based_matrix = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T

In [114]:
common_item_based_matrix.shape

(199, 8371)

In [115]:
item_correlation_df = pd.DataFrame(item_correlation)

In [116]:
item_correlation_df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
item_correlation_df['name'] = df_subtracted.index
item_correlation_df.set_index('name',inplace=True)
item_correlation_df.head()

,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100:Complete First Season (blu-Ray),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"2x Ultra Era with Oxi Booster, 50fl oz",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
list_name = common.name.tolist()

In [119]:
item_correlation_df.columns = df_subtracted.index.tolist()
item_correlation_df_1 =  item_correlation_df[item_correlation_df.index.isin(list_name)]

In [120]:
item_correlation_df_2 = item_correlation_df_1.T[item_correlation_df_1.T.index.isin(list_name)]
item_correlation_df_3 = item_correlation_df_2.T

In [121]:
item_correlation_df_3.head()

,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),4C Grated Parmesan Cheese 100% Natural 8oz Shaker,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Annie's Homegrown Gluten Free Double Chocolate Chip Granola Bars,Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz",...,Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100:Complete First Season (blu-Ray),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Africa's Best No-Lye Dual Conditioning Relaxer System Super,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
item_correlation_df_3[item_correlation_df_3<0]=0

common_item_predicted_ratings = np.dot(item_correlation_df_3, common_item_based_matrix.fillna(0))
common_item_predicted_ratings

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.08080011, 0.06464009, ..., 0.01395515, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.01951687, 0.        , 0.        , ..., 0.        , 0.00155571,
        0.00124457],
       [0.0127492 , 0.        , 0.        , ..., 0.        , 0.00159447,
        0.00127557],
       [0.        , 0.00564345, 0.00451476, ..., 0.        , 0.        ,
        0.        ]])

In [123]:
common_item_predicted_ratings.shape

(199, 8371)

Dummy test will be used for evaluation. To evaluate, we will only make prediction on the items rated by the user. So, this is marked as 1. This is just opposite of dummy_train.

In [124]:
dummy_test = common.copy()
dummy_test['reviews_rating'] = dummy_test['reviews_rating'].apply(lambda x: 1 if x>=1 else 0)
dummy_test = dummy_test.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T.fillna(0)
common_item_predicted_ratings = np.multiply(common_item_predicted_ratings,dummy_test)

The products not rated is marked as 0 for evaluation. And make the item- item matrix representaion.

In [125]:
common_ = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T

In [126]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_item_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

MinMaxScaler(feature_range=(1, 5))
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


C:\Users\kumaraguru\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\kumaraguru\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [127]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [128]:
rmse = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse)

3.559992328750575


In [129]:
train

,id,reviews_username,name,reviews_rating
11155,AVpf3VOfilAPnD_xjpun,gor05751,Clorox Disinfecting Wipes Value Pack Scented 1...,5
22114,AVpf--o9LJeJML43G_Fq,annavsl,Vaseline Intensive Care Healthy Hands Stronger...,4
26218,AVpfPkrkilAPnD_xX-L9,bluenose,"Finish Gelpacs Dishwasher Detergent, Orange Sc...",5
18966,AVpfJP1C1cnluZ0-e3Xy,kiki,Clorox Disinfecting Bathroom Cleaner,5
29999,AVpfW8y_LJeJML437ySW,kcoopxoxo,L'or233al Paris Elvive Extraordinary Clay Reba...,5
...,...,...,...,...
3735,AVpe7sl91cnluZ0-aI1Y,mary,Diet Canada Dry Ginger Ale - 12pk/12 Fl Oz Cans,5
14946,AVpf3VOfilAPnD_xjpun,mwah25,Clorox Disinfecting Wipes Value Pack Scented 1...,5
27518,AVpfRTh1ilAPnD_xYic2,lilym,Planes: Fire Rescue (2 Discs) (includes Digita...,4
26640,AVpfPPkEilAPnD_xX3cP,kim,Aveeno Baby Continuous Protection Lotion Sunsc...,5


In [130]:
test

,id,reviews_username,name,reviews_rating
4332,AVpe9W4D1cnluZ0-avf0,sansname,Hoover174 Platinum Collection153 Lightweight B...,5
3434,AVpe5s3RLJeJML43x4eA,jennypstrong,Pantene Pro-V Expert Collection Age Defy Condi...,5
20634,AVpfkak01cnluZ0-nJj6,girlie,Starbucks Iced Expresso Classics Vanilla Latte...,5
26479,AVpfPPkEilAPnD_xX3cP,kasheville,Aveeno Baby Continuous Protection Lotion Sunsc...,1
29726,AVpfW8y_LJeJML437ySW,payy,L'or233al Paris Elvive Extraordinary Clay Reba...,5
...,...,...,...,...
28375,AVpfrTyiLJeJML43BrSI,bala,Various Artists - Choo Choo Soul (cd),5
20889,AVpfM_ytilAPnD_xXIJb,ronald,Tostitos Bite Size Tortilla Chips,2
3505,AVpe6FfKilAPnD_xQmHi,elunsford,Chex Muddy Buddies Brownie Supreme Snack Mix,5
11361,AVpf3VOfilAPnD_xjpun,meli23melo,Clorox Disinfecting Wipes Value Pack Scented 1...,5


In [133]:
item_final_rating

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.023523,0.000000,0.000000
00sab00,0.0,0.001171,0.0,0.0,0.0,0.0,0.0,0.0,0.003316,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
01impala,0.0,0.048480,0.0,0.0,0.0,0.0,0.0,0.0,0.014725,0.000000,...,0.0,0.0,0.0,0.006729,0.002859,0.0,0.0,0.000000,0.000000,0.003386
0325home,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
06stidriver,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zulaa118,0.0,0.013955,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.039908,0.000000,0.0,0.0,0.000000,0.000000,0.000000
zxcsdfd,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
zxjki,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000534,0.001702,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000622,0.000638,0.000000


#### Identifying the sentiment of the review and picking top 5

In [214]:
df = pd.read_csv("sample30.csv")

#Do reviews classification as good or bad sentiment. 
tfidfFeatures = featuModel.transform(df.reviews_text)
sentClass = clasModel.predict(tfidfFeatures)

In [215]:
#Merge the class to the dataframe
sntmtClassSeries = pd.Series(sentClass, name = "sent_class")
df = df.join(sntmtClassSeries)
print(df[['manufacturer', 'name', 'reviews_text', 'sent_class']])

                             manufacturer  \
0      Universal Music Group / Cash Money   
1                                Lundberg   
2                                Lundberg   
3                                     K-Y   
4                                     K-Y   
...                                   ...   
29995                       L'oreal Paris   
29996                       L'oreal Paris   
29997                       L'oreal Paris   
29998                       L'oreal Paris   
29999                       L'oreal Paris   

                                                    name  \
0              Pink Friday: Roman Reloaded Re-Up (w/dvd)   
1             Lundberg Organic Cinnamon Toast Rice Cakes   
2             Lundberg Organic Cinnamon Toast Rice Cakes   
3                       K-Y Love Sensuality Pleasure Gel   
4                       K-Y Love Sensuality Pleasure Gel   
...                                                  ...   
29995  L'or233al Paris Elvive Extraordi

In [230]:
groupedDf = df.groupby(['manufacturer', 'name'])
product_class = groupedDf['sent_class'].agg(mean_class=np.mean)

In [235]:
print(type(product_class))
print(product_class.info)

<class 'pandas.core.frame.DataFrame'>
<bound method DataFrame.info of                                                                         mean_class
manufacturer        name                                                          
120                 2017-2018 Brownline174 Duraflex 14-Month Planne...    0.750000
                    Craft Punch Giga Scallop Circle 45 24687534 To 334    1.000000
                    Post Bound Jumbo Album - Burgundy (11x14)             1.000000
1200                Storkcraft Tuscany Glider and Ottoman, Beige Cu...    0.981061
20th Century Fox    There's Something About Mary (dvd)                    0.976923
...                                                                            ...
Yes To Inc.         Yes To Carrots Nourishing Body Wash                   1.000000
Yes To, Inc         Yes To Grapefruit Rejuvenating Body Wash              0.846154
York Wallcoverings  Roommates No Place Like Home Peel Stick Wall De...    1.000000
Yosemite Home Dec

In [239]:
product_class.index

MultiIndex([(                          '120', ...),
            (                          '120', ...),
            (                          '120', ...),
            (                         '1200', ...),
            (             '20th Century Fox', ...),
            ('AUSTRALIAN GOLD PRODUCTS LTD.', ...),
            (              'Advanced Beauty', ...),
            (                  'Allure Home', ...),
            (               'AmazonUs/CLOO7', ...),
            (  'American Coffee Partnership', ...),
            ...
            (             'William H Harvey', ...),
            (                       'Wilton', ...),
            (                       'Windex', ...),
            (                          'Wmu', ...),
            (                        'Xenon', ...),
            (                  'Yes To Inc.', ...),
            (                  'Yes To, Inc', ...),
            (           'York Wallcoverings', ...),
            (          'Yosemite Home Decor', ..

In [250]:
for x in product_class.index:
    print(product_class.loc[x])

KeyError: "None of [Index(['120', '2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black'], dtype='object')] are in the [columns]"

In [228]:
groupedDf.get_group(('120','Craft Punch Giga Scallop Circle 45 24687534 To 334'))

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment,sent_class
1262,AV1YpiJvvKc47QAVguxy,No Brand,"Home,Arts & Crafts,Scrapbooking,Scrapbooking T...",120,Craft Punch Giga Scallop Circle 45 24687534 To...,2017-05-08T17:54:36.000Z,NaN,True,5,Very easy on the hands to use. I used it on ca...,Great Punch,NaN,NaN,j9,Positive,1


In [229]:
for rec in groupedDf:
    print(rec)

(('120', '2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black'),                          id      brand  \
28795  AVpfVMhR1cnluZ0-ixY8  Brownline   
28796  AVpfVMhR1cnluZ0-ixY8  Brownline   
28797  AVpfVMhR1cnluZ0-ixY8  Brownline   
28798  AVpfVMhR1cnluZ0-ixY8  Brownline   

                                              categories manufacturer  \
28795  School & Office Supplies,Calendars & Planners,...          120   
28796  School & Office Supplies,Calendars & Planners,...          120   
28797  School & Office Supplies,Calendars & Planners,...          120   
28798  School & Office Supplies,Calendars & Planners,...          120   

                                                    name  \
28795  2017-2018 Brownline174 Duraflex 14-Month Plann...   
28796  2017-2018 Brownline174 Duraflex 14-Month Plann...   
28797  2017-2018 Brownline174 Duraflex 14-Month Plann...   
28798  2017-2018 Brownline174 Duraflex 14-Month Plann...   

                   reviews_date reviews_did

[89 rows x 16 columns])
(('Aveeno Baby', 'Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz'),                          id   brand  \
26448  AVpfPPkEilAPnD_xX3cP  Aveeno   
26449  AVpfPPkEilAPnD_xX3cP  Aveeno   
26450  AVpfPPkEilAPnD_xX3cP  Aveeno   
26451  AVpfPPkEilAPnD_xX3cP  Aveeno   
26452  AVpfPPkEilAPnD_xX3cP  Aveeno   
...                     ...     ...   
26756  AVpfPPkEilAPnD_xX3cP  Aveeno   
26757  AVpfPPkEilAPnD_xX3cP  Aveeno   
26758  AVpfPPkEilAPnD_xX3cP  Aveeno   
26759  AVpfPPkEilAPnD_xX3cP  Aveeno   
26760  AVpfPPkEilAPnD_xX3cP  Aveeno   

                                              categories manufacturer  \
26448  Personal Care,Sun Care,Featured Brands,Health ...  Aveeno Baby   
26449  Personal Care,Sun Care,Featured Brands,Health ...  Aveeno Baby   
26450  Personal Care,Sun Care,Featured Brands,Health ...  Aveeno Baby   
26451  Personal Care,Sun Care,Featured Brands,Health ...  Aveeno Baby   
26452  Personal Care,Sun Care,Featured

[87 rows x 16 columns])
(('Broken Bow', "Jason Aldean - They Don't Know"),                          id               brand  \
26857  AVpfR5m0LJeJML436K3W  Warner Music Group   
26858  AVpfR5m0LJeJML436K3W  Warner Music Group   
26859  AVpfR5m0LJeJML436K3W  Warner Music Group   
26860  AVpfR5m0LJeJML436K3W  Warner Music Group   
26861  AVpfR5m0LJeJML436K3W  Warner Music Group   
...                     ...                 ...   
27056  AVpfR5m0LJeJML436K3W  Warner Music Group   
27057  AVpfR5m0LJeJML436K3W  Warner Music Group   
27058  AVpfR5m0LJeJML436K3W  Warner Music Group   
27059  AVpfR5m0LJeJML436K3W  Warner Music Group   
27060  AVpfR5m0LJeJML436K3W  Warner Music Group   

                                              categories manufacturer  \
26857  Music on CD or Vinyl,Country Music on CD or Vi...   Broken Bow   
26858  Music on CD or Vinyl,Country Music on CD or Vi...   Broken Bow   
26859  Music on CD or Vinyl,Country Music on CD or Vi...   Broken Bow   
26860  Music on CD o

1219                  NaN             ally       Positive           1  )
(('Cetaphil', 'Cetaphil Gentle Skin Cleanser, 2oz'),                         id     brand  \
1369  AV1ZSp2uglJLPUi8IQFy  Cetaphil   
1370  AV1ZSp2uglJLPUi8IQFy  Cetaphil   
1371  AV1ZSp2uglJLPUi8IQFy  Cetaphil   
1372  AV1ZSp2uglJLPUi8IQFy  Cetaphil   
1373  AV1ZSp2uglJLPUi8IQFy  Cetaphil   
1374  AV1ZSp2uglJLPUi8IQFy  Cetaphil   
1375  AV1ZSp2uglJLPUi8IQFy  Cetaphil   

                                             categories manufacturer  \
1369  Personal Care,Skin Care,Moisturizer,Beauty,Fac...     Cetaphil   
1370  Personal Care,Skin Care,Moisturizer,Beauty,Fac...     Cetaphil   
1371  Personal Care,Skin Care,Moisturizer,Beauty,Fac...     Cetaphil   
1372  Personal Care,Skin Care,Moisturizer,Beauty,Fac...     Cetaphil   
1373  Personal Care,Skin Care,Moisturizer,Beauty,Fac...     Cetaphil   
1374  Personal Care,Skin Care,Moisturizer,Beauty,Fac...     Cetaphil   
1375  Personal Care,Skin Care,Moisturizer,Beauty,

6478             anon       Positive           1  )
(('Disguise', "Toy Story Kids' Woody Accessory Kit"),                         id      brand  \
3472  AVpe6C3WLJeJML43yApY  Toy Story   
3473  AVpe6C3WLJeJML43yApY  Toy Story   
3474  AVpe6C3WLJeJML43yApY  Toy Story   
3475  AVpe6C3WLJeJML43yApY  Toy Story   

                                             categories manufacturer  \
3472  Holiday Shop,Halloween,Halloween Costume Acces...     Disguise   
3473  Holiday Shop,Halloween,Halloween Costume Acces...     Disguise   
3474  Holiday Shop,Halloween,Halloween Costume Acces...     Disguise   
3475  Holiday Shop,Halloween,Halloween Costume Acces...     Disguise   

                                     name              reviews_date  \
3472  Toy Story Kids' Woody Accessory Kit      2010-09-18T00:00:00Z   
3473  Toy Story Kids' Woody Accessory Kit      2010-10-13T00:00:00Z   
3474  Toy Story Kids' Woody Accessory Kit  2011-01-07T00:39:52.000Z   
3475  Toy Story Kids' Woody Accessory Kit  

...

18471       Positive           1  )
(('South Bend', 'Plano Mini-Magnum 13-Compartment Tackle Box'),                         id  brand  \
4126  AVpe9vHs1cnluZ0-a4TU  Plano   
4127  AVpe9vHs1cnluZ0-a4TU  Plano   
4128  AVpe9vHs1cnluZ0-a4TU  Plano   
4129  AVpe9vHs1cnluZ0-a4TU  Plano   
4130  AVpe9vHs1cnluZ0-a4TU  Plano   

                                             categories manufacturer  \
4126  Sports & Outdoors,Outdoor Sports,Fishing,Fishi...   South Bend   
4127  Sports & Outdoors,Outdoor Sports,Fishing,Fishi...   South Bend   
4128  Sports & Outdoors,Outdoor Sports,Fishing,Fishi...   South Bend   
4129  Sports & Outdoors,Outdoor Sports,Fishing,Fishi...   South Bend   
4130  Sports & Outdoors,Outdoor Sports,Fishing,Fishi...   South Bend   

                                             name              reviews_date  \
4126  Plano Mini-Magnum 13-Compartment Tackle Box                       NaN   
4127  Plano Mini-Magnum 13-Compartment Tackle Box  2016-12-18T00:00:00.000Z   
4128  P

1449             mary       Positive           1  )
(('Tonic Studios', 'Tim Holtz Retractable Craft Pick-Red 6x.5'),                          id      brand  \
26221  AVpfPKzxLJeJML435SYL  Tim Holtz   
26222  AVpfPKzxLJeJML435SYL  Tim Holtz   
26223  AVpfPKzxLJeJML435SYL  Tim Holtz   

                                              categories   manufacturer  \
26221  Home,Arts & Crafts,Scrapbooking,Scrapbooking T...  Tonic Studios   
26222  Home,Arts & Crafts,Scrapbooking,Scrapbooking T...  Tonic Studios   
26223  Home,Arts & Crafts,Scrapbooking,Scrapbooking T...  Tonic Studios   

                                            name              reviews_date  \
26221  Tim Holtz Retractable Craft Pick-Red 6x.5  2017-06-09T20:01:15.000Z   
26222  Tim Holtz Retractable Craft Pick-Red 6x.5  2017-03-07T00:00:00.000Z   
26223  Tim Holtz Retractable Craft Pick-Red 6x.5  2017-02-03T00:00:00.000Z   

      reviews_didPurchase reviews_doRecommend  reviews_rating  \
26221                 NaN          

3252       Positive           1  )
(("Udi's", "Udi's Pepperoni Pizza"),                         id  brand  \
4518  AVpe-YAL1cnluZ0-bHGh  Udi's   

                                             categories manufacturer  \
4518  Food,Packaged Foods,Frozen Foods,Frozen Meals,...        Udi's   

                       name              reviews_date reviews_didPurchase  \
4518  Udi's Pepperoni Pizza  2016-02-03T05:26:24.000Z                 NaN   

     reviews_doRecommend  reviews_rating  \
4518                 NaN               1   

                                           reviews_text reviews_title  \
4518  This was the WORST pizza I've ever had in my l...    Disgusting   

     reviews_userCity reviews_userProvince reviews_username user_sentiment  \
4518         Stockton                  NaN            bekah       Positive   

      sent_class  
4518           1  )
(('Unfi', 'Noosa Honey Yogurt'),                          id  brand  \
20786  AVpflENxilAPnD_xeXFF  Noosa   

           

18531                  NaN                vickie       Positive           1  )
(('Wagan', 'Wagan Smartac 80watt Inverter With Usb'),                         id  brand  \
3647  AVpe6PCDLJeJML43yFQH  Wagan   
3648  AVpe6PCDLJeJML43yFQH  Wagan   
3649  AVpe6PCDLJeJML43yFQH  Wagan   
3650  AVpe6PCDLJeJML43yFQH  Wagan   
3651  AVpe6PCDLJeJML43yFQH  Wagan   

                                             categories manufacturer  \
3647  Home,Home Improvement,Electrical,Pro Audio,Pow...        Wagan   
3648  Home,Home Improvement,Electrical,Pro Audio,Pow...        Wagan   
3649  Home,Home Improvement,Electrical,Pro Audio,Pow...        Wagan   
3650  Home,Home Improvement,Electrical,Pro Audio,Pow...        Wagan   
3651  Home,Home Improvement,Electrical,Pro Audio,Pow...        Wagan   

                                        name              reviews_date  \
3647  Wagan Smartac 80watt Inverter With Usb  2013-06-10T00:00:00.000Z   
3648  Wagan Smartac 80watt Inverter With Usb  2011-07-18T00:00:

22058                  NaN          justjill       Positive           1  )
(('York Wallcoverings', 'Roommates No Place Like Home Peel Stick Wall Decals'),                          id      brand  \
17533  AVpfBeN6LJeJML430rrn  Roommates   

                                              categories        manufacturer  \
17533  Home,Home Decor,Wall Decor,Home Improvement,Pa...  York Wallcoverings   

                                                    name  \
17533  Roommates No Place Like Home Peel Stick Wall D...   

                   reviews_date reviews_didPurchase reviews_doRecommend  \
17533  2017-06-19T00:37:37.000Z                 NaN                True   

       reviews_rating                                       reviews_text  \
17533               5  Shipping time was less than expected which was...   

      reviews_title reviews_userCity reviews_userProvince  \
17533           NaN              NaN                  NaN   

          reviews_username user_sentiment  sent_cla

In [139]:
d

name
Chester's Cheese Flavored Puffcorn Snacks                                         0.546107
Windex Original Glass Cleaner Refill 67.6oz (2 Liter)                             0.167279
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo                          0.086751
Pleasant Hearth 1,800 sq ft Wood Burning Stove with Blower, Medium, LWS-127201    0.042787
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)               0.032291
Nexxus Exxtra Gel Style Creation Sculptor                                         0.030927
Aussie Aussome Volume Shampoo, 13.5 Oz                                            0.028226
Mike Dave Need Wedding Dates (dvd + Digital)                                      0.027196
Diet Canada Dry Ginger Ale - 12pk/12 Fl Oz Cans                                   0.023710
Dark Shadows (includes Digital Copy) (ultraviolet) (dvdvideo)                     0.021194
Cheetos Crunchy Flamin' Hot Cheese Flavored Snacks                                0.0

In [148]:
for itmName in list(d.index):
    print(itmName)
    Xfeatu = featuModel.transform([itmName])
    print(clasModel.predict(Xfeatu)))

Chester's Cheese Flavored Puffcorn Snacks
[0]
Windex Original Glass Cleaner Refill 67.6oz (2 Liter)
[1]
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo
[1]
Pleasant Hearth 1,800 sq ft Wood Burning Stove with Blower, Medium, LWS-127201
[1]
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)
[1]
Nexxus Exxtra Gel Style Creation Sculptor
[1]
Aussie Aussome Volume Shampoo, 13.5 Oz
[1]
Mike Dave Need Wedding Dates (dvd + Digital)
[1]
Diet Canada Dry Ginger Ale - 12pk/12 Fl Oz Cans
[1]
Dark Shadows (includes Digital Copy) (ultraviolet) (dvdvideo)
[1]
Cheetos Crunchy Flamin' Hot Cheese Flavored Snacks
[1]
Yes To Carrots Nourishing Body Wash
[1]
Coty Airspun Face Powder, Translucent Extra Coverage
[1]
Red (special Edition) (dvdvideo)
[1]
Stargate (ws) (ultimate Edition) (director's Cut) (dvdvideo)
[0]
Newman's Own Balsamic Vinaigrette, 16.0oz
[1]
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd
[1]
Pleasant Hearth Diamond Fireplace Screen - Espresso
[1]
